In [6]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
import tempfile
import os
from joblib import load, dump
import shutil

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
testset = datasets.MNIST('./data', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = len(trainset))
testloader = torch.utils.data.DataLoader(testset, batch_size = len(testset))
trainset_array1 = next(iter(trainloader))[0].numpy()
testset_array1 = next(iter(testloader))[0].numpy()

trainset_labels_array1 = next(iter(trainloader))[1].numpy()
testset_labels_array1 = next(iter(testloader))[1].numpy()

trainset_array = trainset_array1.reshape(60000,784)
testset_array = testset_array1.reshape(10000,784)


In [7]:
import operator
from functools import reduce

In [3]:
start = datetime.now()

temp_folder_test = tempfile.mkdtemp()
filename_test = os.path.join(temp_folder_test, 'test.mmap')
if os.path.exists(filename_test): os.unlink(filename_test)
_ = dump(testset_array, filename_test)
test_memmap = load(filename_test, mmap_mode='r+')

end = datetime.now()
time_taken = end - start
print('Time(test_memmap): ',time_taken)

#########################################

# start = datetime.now()

# temp_folder_train = tempfile.mkdtemp()
# filename_train = os.path.join(temp_folder_train, 'train.mmap')
# if os.path.exists(filename_train): os.unlink(filename_train)
# _ = dump(trainset_array, filename_train)
# train_memmap = load(filename_train, mmap_mode='r+')

# end = datetime.now()
# time_taken = end - start
# print('Time(train_memmap): ',time_taken)

Time(test_memmap):  0:00:00.023589


In [ ]:
del testset_array
# del trainset_array
import gc
_ = gc.collect()

test_memmap_small = test_memmap[:512]
# train_memmap_small = train_memmap[:512]

test_memmap_final = np.asarray(test_memmap_small)
# train_memmap_final = np.asarray(train_memmap_small)

In [10]:
testset_array2 = testset_array[:1024]
trainset_array2 = trainset_array[:1024]
print(testset_array2.shape)

(1024, 784)


In [19]:
def case2(a):
    start = datetime.now()
    list1 = []
    for y in trainset_array2:
        val = sum((a-y)**2)
        list1.append(val)
    ans = trainset_labels_array1[list1.index(min(list1))]         
#     return trainset_labels_array1[np.argmin(list1)] 
    end = datetime.now()
    t = end-start
    return (ans,t)

### Non-parallel

In [ ]:
list_nonp = []
time_nonp = []

In [ ]:
################
# non-parallel #
################

def nonp(testset):
    start = datetime.now()
    list01 = []
    for x in testset:
        list01.append(case2(x))
    end = datetime.now()
    time_taken = end - start
    list_nonp.append(list01)
    time_nonp.append(time_taken)

In [ ]:
def serial_testset_split(x):
    testset_new = np.split(testset_array2,x)
    for x in testset_new:
        nonp(x)

In [ ]:
list_nonp = []
time_nonp = []
serial_testset_split(256)
list_nonp = np.asarray(list_nonp)
print(list_nonp.shape)
print(reduce(operator.add, time_nonp))

In [ ]:
list_nonp

#### Without split

In [5]:
start1 = datetime.now()
l1 = []
for x in testset_array2:
    l1.append(case2(x))
end1 = datetime.now()
timetaken = end1-start1
print(timetaken)

0:00:38.622740


In [ ]:
l1 = np.asarray(l1)
l1.shape

In [ ]:
# l1

## Parallel

### with internal argmin

In [42]:
start1 = datetime.now()

final_list_A = Parallel(n_jobs=2,backend='multiprocessing',batch_size=512)(delayed(case2)(x) for x in testset_array2)

end1 = datetime.now()
time_taken = end1 - start1
print('Time: ',time_taken) 
final_list_A = np.asarray(final_list_A)
print(final_list_A.shape)

My calculation: 100%|██████████| 1024/1024 [01:19<00:00, 12.82it/s]

Time:  0:01:19.851987
(1024, 2)


In [17]:
# try:
#     shutil.rmtree(temp_folder_test)
# #     shutil.rmtree(temp_folder_train)
# except OSError:
#     pass

In [21]:
final_list_A

array([[7, datetime.timedelta(microseconds=358925)],
       [2, datetime.timedelta(microseconds=356413)],
       [1, datetime.timedelta(microseconds=346385)],
       ...,
       [7, datetime.timedelta(microseconds=170330)],
       [3, datetime.timedelta(microseconds=166208)],
       [4, datetime.timedelta(microseconds=158378)]], dtype=object)

In [22]:
time1 = []
for x in final_list_A:
    time1.append(x[1])
time1

[datetime.timedelta(microseconds=358925),
 datetime.timedelta(microseconds=356413),
 datetime.timedelta(microseconds=346385),
 datetime.timedelta(microseconds=363588),
 datetime.timedelta(microseconds=352121),
 datetime.timedelta(microseconds=359427),
 datetime.timedelta(microseconds=348612),
 datetime.timedelta(microseconds=355356),
 datetime.timedelta(microseconds=358383),
 datetime.timedelta(microseconds=353827),
 datetime.timedelta(microseconds=355625),
 datetime.timedelta(microseconds=361430),
 datetime.timedelta(microseconds=348095),
 datetime.timedelta(microseconds=358795),
 datetime.timedelta(microseconds=351091),
 datetime.timedelta(microseconds=359716),
 datetime.timedelta(microseconds=371549),
 datetime.timedelta(microseconds=359896),
 datetime.timedelta(microseconds=361407),
 datetime.timedelta(microseconds=361492),
 datetime.timedelta(microseconds=356434),
 datetime.timedelta(microseconds=361761),
 datetime.timedelta(microseconds=364180),
 datetime.timedelta(microseconds=3

In [23]:
tl1= []
for x in time1:
    tl1.append(x.microseconds)

In [24]:
tl1

[358925,
 356413,
 346385,
 363588,
 352121,
 359427,
 348612,
 355356,
 358383,
 353827,
 355625,
 361430,
 348095,
 358795,
 351091,
 359716,
 371549,
 359896,
 361407,
 361492,
 356434,
 361761,
 364180,
 365167,
 366377,
 365039,
 363459,
 364011,
 361653,
 353141,
 364792,
 361240,
 357015,
 358304,
 363003,
 365139,
 363996,
 369399,
 361573,
 367726,
 358910,
 365596,
 477551,
 417129,
 405479,
 401578,
 406767,
 404202,
 404669,
 402690,
 406373,
 401175,
 360259,
 393713,
 400120,
 406525,
 401090,
 408725,
 413466,
 405268,
 414520,
 406493,
 405957,
 420905,
 359080,
 370473,
 373384,
 362968,
 359759,
 358973,
 362509,
 363994,
 365918,
 364807,
 367650,
 363218,
 367712,
 408435,
 416039,
 421542,
 420725,
 430501,
 427625,
 423826,
 424632,
 429982,
 423832,
 431994,
 358011,
 356664,
 363137,
 359898,
 356578,
 358549,
 361115,
 358464,
 362780,
 369510,
 360870,
 361133,
 372041,
 387363,
 461163,
 384999,
 370219,
 367195,
 381123,
 375078,
 385717,
 372458,
 371583,
 

In [25]:
sum(tl1)/1000000

392.973583

In [26]:
tl1 = np.asarray(tl1)

In [35]:
tl2 = np.split(tl1,2)

In [36]:
tl2 = np.asarray(tl2)
tl2.shape

(2, 512)

In [37]:
tl2

array([[358925, 356413, 346385, ..., 388487, 383397, 390153],
       [380799, 377625, 450114, ..., 170330, 166208, 158378]])

### with external argmin

In [ ]:
def case2B(a):
    list1 = []
    for y in train_memmap_final:
        val = sum((a-y)**2)
        list1.append(val)
    return list1

In [ ]:
start = datetime.now()

final_list = Parallel(n_jobs=8,backend='multiprocessing',batch_size=64,max_nbytes=None)(delayed(case2B)(x) for x in test_memmap_final)
final_list_B = []
i=0
for x in final_list:
    y = np.argmin(x)
    final_list_B.append(trainset_labels_array1[y])
    i+=1
end = datetime.now()
time_taken = end - start
print('Time: ',time_taken) 
final_list_B = np.asarray(final_list_B)
print(final_list_B.shape)

In [ ]:
final_list_B

In [ ]:
testset_array3 = testset_array2[:256]
testset_array4 = testset_array2[256:512]
print(testset_array3.shape)
print(testset_array4.shape)

## Accuracy:

In [ ]:
testset_labels_array1 = testset_labels_array1[:512]

In [ ]:
j = 0
for x in range(0,512):
    if final_list[x] == testset_labels_array1[x]:
        j += 1
j

In [ ]:
403/512

In [ ]:
print(final_list[0])
print(testset_labels_array1[0])

In [ ]:
testimg = []
for i in range(1,513):
    testimg.append(i)
    

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = testimg
y = tl1
ax.bar(x,y)
plt.show()